# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os

# Import API key
from api_keys import geoapify_key

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,avarua,-21.2078,-159.7750,300.18,83,75,12.35,CK,1702244412
1,1,edinburgh of the seven seas,-37.0676,-12.3116,286.80,69,97,6.48,SH,1702244413
2,2,ushuaia,-54.8000,-68.3000,280.96,57,0,8.75,AR,1702244274
3,3,grytviken,-54.2811,-36.5092,278.77,90,100,6.68,GS,1702244276
4,4,kimbe,-5.5502,150.1429,297.95,97,100,1.17,PG,1702244415


In [10]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(x='Lng', 
                                      y='Lat',
                                      geo=True,
                                      tiles='OSM',
                                      frame_width = 800,
                                      frame_height = 500,
                                      size='Humidity',
                                      title='Map of Cities with Humidity',
                                      xlabel='Longitude', 
                                      ylabel='Latitude',
                                      color='Humidity',
                                      hover_cols=['City', 'Humidity'],
                                      alpha=0.5)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions = (city_data_df["Max Temp"] > 70) & (city_data_df["Max Temp"] < 90) & \
                   (city_data_df["Wind Speed"] < 10) & \
                   (city_data_df["Cloudiness"] == 10)

# Narrow down cities that fit the ideal conditions
ideal_cities_df = city_data_df[ideal_conditions].copy()

# Drop any rows with null values
ideal_cities_df.dropna(inplace=True)

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "filter": "category=hotels",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["bias"] = f"proximity,{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


   # Make and API request using the params dictionaty
response = requests.get(base_url, params=params)
    
# Convert the API response to JSON format
name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

IndentationError: unexpected indent (4077823898.py, line 31)

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(x='Lng', 
                                      y='Lat',
                                      geo=True,
                                      tiles='OSM',
                                      frame_width=800,
                                      frame_height=500,
                                      size='Humidity',
                                      title='City Humidity',
                                      xlabel='Longitude', 
                                      ylabel='Latitude',
                                      color='Humidity',
                                      hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'],
                                      alpha=0.5)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)